In [1]:
import socket
import struct
import numpy as np
import time
import threading

class TeensyUDPBridge():
    def __init__(self, 
                 bridge_ip='192.168.123.10', 
                 bridge_port=5000, 
                 host_ip='0.0.0.0', 
                 host_port=5000,
                 user_callback=None):
        self.bridge_ip = bridge_ip
        self.bridge_port = bridge_port
        self.socket=socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
        self.RX_RUNNING = True
        self.socket.bind((host_ip, host_port))
        self.rx_thread=threading.Thread(target=self.receivingThread)
        self.state = None
        self.latest_state_stamp = time.time()
        self.packet = None
        self.user_callback = user_callback
        self.rx_thread.start()

    def sendCommand(self, cmd):
        msg_format=f'{len(cmd)}f'
        payload = cmd
        arguments = [msg_format] + payload
        packet=struct.pack(*arguments)
        self.socket.sendto(packet, (self.bridge_ip, self.bridge_port))

    def getLatestState(self):
        return self.latest_state_stamp, self.state

    def receivingThread(self):
        while self.RX_RUNNING:
            packet = self.socket.recvmsg(4096)[0]
            state_msg_format=f'{len(packet)//4}f'
            data = struct.unpack(state_msg_format, packet)
            self.state = data
            self.latest_state_stamp = time.time()
            # Notify user if a callback is provided
            if self.user_callback is not None:
                self.user_callback(self)

    def close(self):
        self.RX_RUNNING = False
        self.socket.close()
        self.rx_thread.join()

In [2]:
udp_bridge = TeensyUDPBridge()

In [38]:
udp_bridge.getLatestState()

(1737258315.9701846,
 (-14.055112838745117,
  -0.19855839014053345,
  0.33745068311691284,
  -0.0511326938867569,
  -0.0511326938867569,
  0.0,
  0.0087890625,
  0.0048828125,
  -0.00390625))

In [352]:
class SingleFingerRobot:
    def __init__(self, bridge=None, robot_name='finger1', joint_order = ['hip', 'thigh', 'knee'], current2Torque=1.0, gear_ratio = 9, max_torque = 20.0):
        self.robot_name = robot_name    
        if bridge is None:
            assert NotImplementedError, 'Default DDS bridge is not implemented yet'
        else:
            self.bridge = bridge
        self.name2Index = {'hip': 0, 'thigh': 1, 'knee': 2}
        self.index2Name = {v:k for k,v in self.name2Index.items()}
        self.joints_idx = np.array([self.name2Index[j] for j in joint_order])
        self.q_offset = 0
        self.dq_offset = 3
        self.current_offset = 6
        self.current2Torque = current2Torque
        self.max_torque = max_torque    
        self.gear_ratio = gear_ratio 
        self.q_offsets = np.zeros(3)

    def getState(self):
        stamp, state = self.bridge.getLatestState()
        state = np.array(state)
        q = state[self.joints_idx+self.q_offset]/self.gear_ratio - self.q_offsets
        dq = state[self.joints_idx+self.dq_offset]/self.gear_ratio
        torque = state[self.joints_idx+self.current_offset] * self.current2Torque * self.gear_ratio
        if time.time() - stamp > 0.1:
            return None
        else:
            return dict(time = stamp, q = q, dq = dq, torque = torque)
    
    def resetJoints(self):
        if self.getState() is None:
            return False
        else:
            self.q_offsets = np.zeros(3)
            state = self.getState()
            self.q_offsets = state['q'].copy()
            return True
    
    def setTroque(self, torques):
        torques = np.clip(torques, -self.max_torque, self.max_torque)
        current = (np.array(torques)[self.joints_idx]/self.gear_ratio)/self.current2Torque
        cmd = current.tolist() + 12*[0.]
        self.bridge.sendCommand(cmd)

In [353]:
robot = SingleFingerRobot(udp_bridge)

In [354]:
if not robot.resetJoints():
    print('Failed to reset joints')
robot.getState()

{'time': 1737260140.899632,
 'q': array([0., 0., 0.]),
 'dq': array([ 0.        ,  0.        , -0.00568141]),
 'torque': array([-0.11425781, -0.02636719, -0.01757812])}

In [355]:
robot.getState()

{'time': 1737260141.9856563,
 'q': array([0., 0., 0.]),
 'dq': array([ 0.        ,  0.        , -0.00568141]),
 'torque': array([ 0.        , -0.04394531,  0.05273438])}

In [356]:
import time
cmd = 15*[0]
Kp = 80.0
Kv = 0.0
for i in range(5000):
    state = robot.getState()
    q = state['q']
    dq = state['dq']
    knee_q = q[2]
    knee_dq = dq[2]
    torque = Kp * (0.0 - knee_q) - Kv * knee_dq
    torques = np.array([0, 0, torque])
    robot.setTroque(torques)
    time.sleep(0.01)

KeyboardInterrupt: 